In [1]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

In [2]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [4]:
h = 28 
max_lags = 57
tr_last = 1913  # 历史的最后一天
fday = datetime(2016,4, 25)  # 预测的第一天
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [6]:
prices = pd.read_csv("sell_prices.csv", dtype = PRICE_DTYPES)

In [7]:
cal = pd.read_csv("calendar.csv", dtype = CAL_DTYPES)
cal.head(1)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [9]:
dt = pd.read_csv("sales_train_validation.csv")
dt.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1


In [10]:
%config ZMQInteractiveShell.ast_node_interactivity='all'

In [17]:
def create_dt(is_train = True, nrows = None, first_day = 1200):
    prices = pd.read_csv("sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")  # 类似于labelencoder，将store_id和item_id数字化表示
            prices[col] -= prices[col].min()  
            
    cal = pd.read_csv("calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])  #  转化为datetime类型
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")  # 将event_name_1/event_type_1/event_name_2/event_type_2weekday数字化表示
            cal[col] -= cal[col].min()  # NaN会转换为-1，该代表将NaN归为0，其他从1开始
    
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)  # 预测的第一天
    
    # 处理train_validation文件
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]  # 选取历史数据，从first_day到tr_last（包含这一天）d_1200,d_1201
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols}   # d_1200为float32
    dtype.update({col: "category" for col in catcols if col != "id"})  # 除id外，都为category类型
    dt = pd.read_csv("sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)  # 取catcols和numcols的列(只取了需要的历史天数销量数据)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")  # 数字化
            dt[col] -= dt[col].min()
    
    if not is_train:  # 不是训练集
        for day in range(tr_last+1, tr_last+ 28 +1):  # 遍历预测的每一天
            dt[f"d_{day}"] = np.nan  # 先填充为NaN
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return dt

In [18]:
FIRST_DAY = 350
df = create_dt(is_train=True, first_day= FIRST_DAY)

In [19]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):  # [(7, lag_7), (28, lag_28)]
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)  # 新增两列，同一商品前7天的单价和前28天的价格

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean()) 

            # log=7，wins=28表示一个商品当前的参考价为：从当前时间7天前（如1.8号的七天前是1.1号）开始一共向上取28天，求这些天的均值
    
    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [20]:
create_fea(df)

In [21]:
df.tail(10)

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
40718209,FOODS_3_823_WI_3_validation,3044,6,9,2,2,d_1912,0.0,2016-04-23,11613,...,2.98,0.0,0.0,0.571429,0.000000,0.142857,0.285714,16,2,23
40718210,FOODS_3_823_WI_3_validation,3044,6,9,2,2,d_1913,1.0,2016-04-24,11613,...,2.98,0.0,0.0,0.571429,0.000000,0.142857,0.250000,16,2,24
40718211,FOODS_3_824_WI_3_validation,3045,6,9,2,2,d_1912,1.0,2016-04-23,11613,...,2.48,0.0,0.0,0.428571,0.000000,0.285714,0.000000,16,2,23
40718212,FOODS_3_824_WI_3_validation,3045,6,9,2,2,d_1913,0.0,2016-04-24,11613,...,2.48,0.0,0.0,0.285714,0.000000,0.285714,0.000000,16,2,24
40718213,FOODS_3_825_WI_3_validation,3046,6,9,2,2,d_1912,1.0,2016-04-23,11613,...,3.98,1.0,0.0,1.285714,0.571429,0.928571,1.214286,16,2,23
40718214,FOODS_3_825_WI_3_validation,3046,6,9,2,2,d_1913,0.0,2016-04-24,11613,...,3.98,0.0,1.0,1.000000,0.714286,0.928571,1.250000,16,2,24
40718215,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1912,1.0,2016-04-23,11613,...,1.28,0.0,2.0,0.857143,1.142857,1.035714,1.107143,16,2,23
40718216,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1913,3.0,2016-04-24,11613,...,1.28,1.0,4.0,0.714286,1.571429,1.035714,1.250000,16,2,24
40718217,FOODS_3_827_WI_3_validation,3048,6,9,2,2,d_1912,0.0,2016-04-23,11613,...,1.00,0.0,0.0,0.000000,2.285714,1.821429,1.785714,16,2,23
40718218,FOODS_3_827_WI_3_validation,3048,6,9,2,2,d_1913,0.0,2016-04-24,11613,...,1.00,0.0,5.0,0.000000,2.428571,1.678571,1.964286,16,2,24


In [22]:
df.dropna(inplace = True)
df.shape

(39041269, 31)

In [23]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
print(train_cols)
X_train = df[train_cols]
y_train = df["sales"]

Index(['item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'wday', 'month',
       'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_7', 'lag_28',
       'rmean_7_7', 'rmean_28_7', 'rmean_7_28', 'rmean_28_28', 'week',
       'quarter', 'mday'],
      dtype='object')


In [24]:
np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 1_000_000, replace = False)  # 从index中，随机选择1000000个数
print(fake_valid_inds)
print(len(fake_valid_inds))

train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)  # train是valid剩下的id号
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)

[18486699 25172727 11439665 ... 22567259 22713501 21903962]
1000000


In [25]:
del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

35

In [26]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 2000,
    'num_leaves': 128,
    "min_data_in_leaf": 50,
}

In [27]:
m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=100)

/home/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	valid_0's rmse: 2.45087
[200]	valid_0's rmse: 2.41833
[300]	valid_0's rmse: 2.38994
[400]	valid_0's rmse: 2.37049
[500]	valid_0's rmse: 2.35696
[600]	valid_0's rmse: 2.34695
[700]	valid_0's rmse: 2.33639
[800]	valid_0's rmse: 2.32833
[900]	valid_0's rmse: 2.32274
[1000]	valid_0's rmse: 2.31686
[1100]	valid_0's rmse: 2.31129
[1200]	valid_0's rmse: 2.30719
[1300]	valid_0's rmse: 2.30314
[1400]	valid_0's rmse: 2.29838
[1500]	valid_0's rmse: 2.29453
[1600]	valid_0's rmse: 2.29215
[1700]	valid_0's rmse: 2.28969
[1800]	valid_0's rmse: 2.28719
[1900]	valid_0's rmse: 2.28502
[2000]	valid_0's rmse: 2.28325


In [28]:
m_lgb.save_model("model.lgb")

In [29]:
alphas = [1.035, 1.03, 1.025]
weights = [1/len(alphas)]*len(alphas)  # 权重，当前权重值一样
print(weights)

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]


In [30]:
# alphas：利用lgb预测出模型后，乘以alphas才为最终的预测值
# weight：由于有三个alphas，这三个alphas得出的结果各赋予一定的权重

sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):  # [(1.035, 0.333), (1.03, 0.333), (1.025, 0.333)]
    # te是所有的数据，tst是部分数据
    te = create_dt(False)  # is_train = False  原始数据
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)  # 预测的具体哪一天，比如第1914那天的日期
        print(icount, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()  
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]  # 需要预测的数据
        te.loc[te.date == day, "sales"] = alpha * m_lgb.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()  # 取出预测的数据
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]  # [F1,F2,F3,...F28]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()  # 转变成submission的形式（只是index只有每种商品的预测值，没有分位数和汇总的一些）
    te_sub.fillna(0., inplace = True) 
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv",index=False)

0 2016-04-25 00:00:00
0 2016-04-26 00:00:00
0 2016-04-27 00:00:00
0 2016-04-28 00:00:00
0 2016-04-29 00:00:00
0 2016-04-30 00:00:00
0 2016-05-01 00:00:00
0 2016-05-02 00:00:00
0 2016-05-03 00:00:00
0 2016-05-04 00:00:00
0 2016-05-05 00:00:00
0 2016-05-06 00:00:00
0 2016-05-07 00:00:00
0 2016-05-08 00:00:00
0 2016-05-09 00:00:00
0 2016-05-10 00:00:00
0 2016-05-11 00:00:00
0 2016-05-12 00:00:00
0 2016-05-13 00:00:00
0 2016-05-14 00:00:00
0 2016-05-15 00:00:00
0 2016-05-16 00:00:00
0 2016-05-17 00:00:00
0 2016-05-18 00:00:00
0 2016-05-19 00:00:00
0 2016-05-20 00:00:00
0 2016-05-21 00:00:00
0 2016-05-22 00:00:00
0 1.035 0.3333333333333333
1 2016-04-25 00:00:00
1 2016-04-26 00:00:00
1 2016-04-27 00:00:00
1 2016-04-28 00:00:00
1 2016-04-29 00:00:00
1 2016-04-30 00:00:00
1 2016-05-01 00:00:00
1 2016-05-02 00:00:00
1 2016-05-03 00:00:00
1 2016-05-04 00:00:00
1 2016-05-05 00:00:00
1 2016-05-06 00:00:00
1 2016-05-07 00:00:00
1 2016-05-08 00:00:00
1 2016-05-09 00:00:00
1 2016-05-10 00:00:00
1 201

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm as tqdm

from ipywidgets import widgets, interactive, interact
import ipywidgets as widgets
from IPython.display import display

import os

In [33]:
train_sales = pd.read_csv('sales_train_validation.csv')
calendar_df = pd.read_csv('calendar.csv')
submission_file = pd.read_csv('sample_submission.csv')
sell_prices = pd.read_csv('sell_prices.csv')
lgm_sub = pd.read_csv("submission.csv")

In [34]:
submission_file.head(1)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,Total_X_0.005_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
lgm_col = ['id']
lgm_col.extend([f'd_{day}' for day in range(1914, 1914+28)])
lgm_sub.columns = lgm_col

In [36]:
lgm_sub.head(1)
train_sales.head(1)

,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,FOODS_1_001_CA_1_validation,0.894794,0.837135,0.809365,0.805796,1.082426,1.125519,1.226911,1.036959,0.980982,...,1.080162,1.376785,1.239006,0.962037,0.82995,0.8062,0.83345,1.001705,1.244806,1.220596


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1


In [37]:
print(train_sales.shape)
train_sales = train_sales.merge(lgm_sub, on='id')
print(train_sales.shape)

(30490, 1919)
(30490, 1947)


In [38]:
train_sales.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.938297,1.073401,1.116953,0.842907,0.781406,0.778574,0.699717,0.903393,1.130289,1.145977


In [39]:
total = ['Total']
train_sales['Total'] = 'Total'
train_sales['state_cat'] = train_sales.state_id + "_" + train_sales.cat_id
train_sales['state_dept'] = train_sales.state_id + "_" + train_sales.dept_id
train_sales['store_cat'] = train_sales.store_id + "_" + train_sales.cat_id
train_sales['store_dept'] = train_sales.store_id + "_" + train_sales.dept_id
train_sales['state_item'] = train_sales.state_id + "_" + train_sales.item_id
train_sales['item_store'] = train_sales.item_id + "_" + train_sales.store_id

In [40]:
train_sales.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1939,d_1940,d_1941,Total,state_cat,state_dept,store_cat,store_dept,state_item,item_store
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.903393,1.130289,1.145977,Total,CA_HOBBIES,CA_HOBBIES_1,CA_1_HOBBIES,CA_1_HOBBIES_1,CA_HOBBIES_1_001,HOBBIES_1_001_CA_1


In [41]:
val_eval = ['validation', 'evaluation']

# creating lists for different aggregation levels
total = ['Total']
states = ['CA', 'TX', 'WI']
num_stores = [('CA',4), ('TX',3), ('WI',3)]
stores = [x[0] + "_" + str(y + 1) for x in num_stores for y in range(x[1])]
print(stores)  # 商店名

['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']


In [42]:
cats = ['FOODS', 'HOBBIES', 'HOUSEHOLD']
num_depts = [('FOODS',3), ('HOBBIES',2), ('HOUSEHOLD',2)]
depts = [x[0] + "_" + str(y + 1) for x in num_depts for y in range(x[1])]
state_cats = [state + "_" + cat for state in states for cat in cats]
state_depts = [state + "_" + dept for state in states for dept in depts]
store_cats = [store + "_" + cat for store in stores for cat in cats]
store_depts = [store + "_" + dept for store in stores for dept in depts]
print(state_cats)  # 州名+大类
print("=================================================================")
print(store_depts)  # 商店名（含州）+小类（含大类）

['CA_FOODS', 'CA_HOBBIES', 'CA_HOUSEHOLD', 'TX_FOODS', 'TX_HOBBIES', 'TX_HOUSEHOLD', 'WI_FOODS', 'WI_HOBBIES', 'WI_HOUSEHOLD']
['CA_1_FOODS_1', 'CA_1_FOODS_2', 'CA_1_FOODS_3', 'CA_1_HOBBIES_1', 'CA_1_HOBBIES_2', 'CA_1_HOUSEHOLD_1', 'CA_1_HOUSEHOLD_2', 'CA_2_FOODS_1', 'CA_2_FOODS_2', 'CA_2_FOODS_3', 'CA_2_HOBBIES_1', 'CA_2_HOBBIES_2', 'CA_2_HOUSEHOLD_1', 'CA_2_HOUSEHOLD_2', 'CA_3_FOODS_1', 'CA_3_FOODS_2', 'CA_3_FOODS_3', 'CA_3_HOBBIES_1', 'CA_3_HOBBIES_2', 'CA_3_HOUSEHOLD_1', 'CA_3_HOUSEHOLD_2', 'CA_4_FOODS_1', 'CA_4_FOODS_2', 'CA_4_FOODS_3', 'CA_4_HOBBIES_1', 'CA_4_HOBBIES_2', 'CA_4_HOUSEHOLD_1', 'CA_4_HOUSEHOLD_2', 'TX_1_FOODS_1', 'TX_1_FOODS_2', 'TX_1_FOODS_3', 'TX_1_HOBBIES_1', 'TX_1_HOBBIES_2', 'TX_1_HOUSEHOLD_1', 'TX_1_HOUSEHOLD_2', 'TX_2_FOODS_1', 'TX_2_FOODS_2', 'TX_2_FOODS_3', 'TX_2_HOBBIES_1', 'TX_2_HOBBIES_2', 'TX_2_HOUSEHOLD_1', 'TX_2_HOUSEHOLD_2', 'TX_3_FOODS_1', 'TX_3_FOODS_2', 'TX_3_FOODS_3', 'TX_3_HOBBIES_1', 'TX_3_HOBBIES_2', 'TX_3_HOUSEHOLD_1', 'TX_3_HOUSEHOLD_2', 'WI_

In [43]:
prods = list(train_sales.item_id.unique())  # 商品列表
prod_state = [prod + "_" + state for prod in prods for state in states]  # 商品（含大类和小类）+州名
prod_store = [prod + "_" + store for prod in prods for store in stores]  # 商品（含大类和小类）+商店名（含州）

In [44]:
print("Departments: ", depts)
print("Categories by state: ", state_cats)

Departments:  ['FOODS_1', 'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2']
Categories by state:  ['CA_FOODS', 'CA_HOBBIES', 'CA_HOUSEHOLD', 'TX_FOODS', 'TX_HOBBIES', 'TX_HOUSEHOLD', 'WI_FOODS', 'WI_HOBBIES', 'WI_HOUSEHOLD']


In [45]:
quants = ['0.005', '0.025', '0.165', '0.250', '0.500', '0.750', '0.835', '0.975', '0.995']
days = range(1, 1913 + 29)
time_series_columns = [f'd_{i}' for i in days]

In [46]:
time_series_columns[-20:]

['d_1922',
 'd_1923',
 'd_1924',
 'd_1925',
 'd_1926',
 'd_1927',
 'd_1928',
 'd_1929',
 'd_1930',
 'd_1931',
 'd_1932',
 'd_1933',
 'd_1934',
 'd_1935',
 'd_1936',
 'd_1937',
 'd_1938',
 'd_1939',
 'd_1940',
 'd_1941']

In [47]:
train_sales.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1939,d_1940,d_1941,Total,state_cat,state_dept,store_cat,store_dept,state_item,item_store
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.903393,1.130289,1.145977,Total,CA_HOBBIES,CA_HOBBIES_1,CA_1_HOBBIES,CA_1_HOBBIES_1,CA_HOBBIES_1_001,HOBBIES_1_001_CA_1


In [48]:
print(state_cats)

['CA_FOODS', 'CA_HOBBIES', 'CA_HOUSEHOLD', 'TX_FOODS', 'TX_HOBBIES', 'TX_HOUSEHOLD', 'WI_FOODS', 'WI_HOBBIES', 'WI_HOUSEHOLD']


In [49]:
def create_sales(name_list, group):
    '''
    This function returns a dataframe (sales) on the aggregation level given by name list and group
    '''
    # rows_ve = [(name + "_X_" + str(q) + "_" + ve, str(q)) for name in name_list for q in quants for ve in val_eval]
    sales = train_sales.groupby(group)[time_series_columns].sum() #would not be necessary for lowest level
    return sales

In [50]:
def create_quantile_dict(name_list = stores, group = 'store_id' ,X = False):
    '''
    This function writes creates sales data on given aggregation level, and then writes predictions to the global dictionary my_dict
    '''
    sales = create_sales(name_list, group)
    sales = sales.iloc[:, 1857:] #using the last few months data only
    sales_quants = pd.DataFrame(index = sales.index)  # 空ataFrame，初始化为空
    for q in quants:
        sales_quants[q] = np.quantile(sales, float(q), axis = 1)  # 增加分位数的列，如CA_FOODS计算196天的0.005分位数
    full_mean = pd.DataFrame(np.mean(sales, axis = 1))  #  新的DataFrame，只有一列，表示每个index最近196天的均值
    daily_means = pd.DataFrame(index = sales.index)  # 新的DataFrame，初始化为空
    for i in range(7):
        daily_means[str(i)] = np.mean(sales.iloc[:, i::7], axis = 1)   # 新增7列，一周内每天的均值，如最近196天CA_FOOD,周一的均值
        
    daily_factors = daily_means / np.array(full_mean)
    
    daily_factors = pd.concat([daily_factors, daily_factors, daily_factors, daily_factors], axis = 1)
    daily_factors_np = np.array(daily_factors)

    factor_df = pd.DataFrame(daily_factors_np, columns = submission_file.columns[1:])
    factor_df.index = daily_factors.index

    for i,x in enumerate(tqdm(sales_quants.index)):
        for q in quants:
            v = sales_quants.loc[x, q] * np.array(factor_df.loc[x, :])
            if X:
                my_dict[x + "_X_" + q + "_validation"] = v
                my_dict[x + "_X_" + q + "_evaluation"] = v
            else:
                my_dict[x + "_" + q + "_validation"] = v
                my_dict[x + "_" + q + "_evaluation"] = v

In [51]:
my_dict = {}
create_quantile_dict(state_cats, 'state_cat') #6
print(my_dict)


{'CA_FOODS_0.005_validation': array([ 8852.11677766,  8222.20333685,  7875.04881775,  7875.31750771,
        8945.93076941, 11195.06448828, 11546.63330234,  8852.11677766,
        8222.20333685,  7875.04881775,  7875.31750771,  8945.93076941,
       11195.06448828, 11546.63330234,  8852.11677766,  8222.20333685,
        7875.04881775,  7875.31750771,  8945.93076941, 11195.06448828,
       11546.63330234,  8852.11677766,  8222.20333685,  7875.04881775,
        7875.31750771,  8945.93076941, 11195.06448828, 11546.63330234]), 'CA_FOODS_0.005_evaluation': array([ 8852.11677766,  8222.20333685,  7875.04881775,  7875.31750771,
        8945.93076941, 11195.06448828, 11546.63330234,  8852.11677766,
        8222.20333685,  7875.04881775,  7875.31750771,  8945.93076941,
       11195.06448828, 11546.63330234,  8852.11677766,  8222.20333685,
        7875.04881775,  7875.31750771,  8945.93076941, 11195.06448828,
       11546.63330234,  8852.11677766,  8222.20333685,  7875.04881775,
        7875.31

In [52]:
my_dict = {}
#adding prediction to my_dict on all 12 aggregation levels
create_quantile_dict(total, 'Total', X=True) #1
create_quantile_dict(states, 'state_id', X=True) #2
create_quantile_dict(stores, 'store_id', X=True) #3
create_quantile_dict(cats, 'cat_id', X=True) #4
create_quantile_dict(depts, 'dept_id', X=True) #5
create_quantile_dict(state_cats, 'state_cat') #6
create_quantile_dict(state_depts, 'state_dept') #7
create_quantile_dict(store_cats, 'store_cat') #8
create_quantile_dict(store_depts, 'store_dept') #9
create_quantile_dict(prods, 'item_id', X=True) #10
create_quantile_dict(prod_state, 'state_item') #11
create_quantile_dict(prod_store, 'item_store') #12

In [53]:
pred_df = pd.DataFrame(my_dict)
pred_df = pred_df.transpose()
pred_df_reset = pred_df.reset_index()
final_pred = pd.merge(pd.DataFrame(submission_file.id), pred_df_reset, left_on = 'id', right_on = 'index')
del final_pred['index']
final_pred = final_pred.rename(columns={0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7', 7: 'F8', 8: 'F9',
                                        9: 'F10', 10: 'F11', 11: 'F12', 12: 'F13', 13: 'F14', 14: 'F15', 15: 'F16',
                                        16: 'F17', 17: 'F18', 18: 'F19', 19: 'F20', 20: 'F21', 21: 'F22', 
                                        22: 'F23', 23: 'F24', 24: 'F25', 25: 'F26', 26: 'F27', 27: 'F28'})

In [54]:
final_pred = final_pred.fillna(0)

In [55]:
final_pred

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,Total_X_0.005_validation,31156.041519,29445.790060,28389.674310,28912.127334,32920.351131,40022.738609,40017.967037,31156.041519,29445.790060,...,32920.351131,40022.738609,40017.967037,31156.041519,29445.790060,28389.674310,28912.127334,32920.351131,40022.738609,40017.967037
1,CA_X_0.005_validation,13612.623476,12640.163661,12155.006590,12195.437047,13823.827635,17475.929625,17922.191966,13612.623476,12640.163661,...,13823.827635,17475.929625,17922.191966,13612.623476,12640.163661,12155.006590,12195.437047,13823.827635,17475.929625,17922.191966
2,TX_X_0.005_validation,8187.544756,7780.862334,7459.299876,7706.693008,8420.975894,9854.580057,10313.974075,8187.544756,7780.862334,...,8420.975894,9854.580057,10313.974075,8187.544756,7780.862334,7459.299876,7706.693008,8420.975894,9854.580057,10313.974075
3,WI_X_0.005_validation,8477.493308,8171.897386,7946.256653,8152.786642,9665.853843,11513.064749,10688.772418,8477.493308,8171.897386,...,9665.853843,11513.064749,10688.772418,8477.493308,8171.897386,7946.256653,8152.786642,9665.853843,11513.064749,10688.772418
4,CA_1_X_0.005_validation,3170.109533,2823.554452,2730.739498,2775.387941,3277.519495,4248.228170,4306.510911,3170.109533,2823.554452,...,3277.519495,4248.228170,4306.510911,3170.109533,2823.554452,2730.739498,2775.387941,3277.519495,4248.228170,4306.510911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771115,FOODS_3_823_WI_3_0.995_evaluation,2.612039,0.822642,2.610064,1.458018,2.090875,2.202822,2.203540,2.612039,0.822642,...,2.090875,2.202822,2.203540,2.612039,0.822642,2.610064,1.458018,2.090875,2.202822,2.203540
771116,FOODS_3_824_WI_3_0.995_evaluation,2.726973,1.347625,0.697624,0.724040,0.714887,2.727823,2.156027,2.726973,1.347625,...,0.714887,2.727823,2.156027,2.726973,1.347625,0.697624,0.724040,0.714887,2.727823,2.156027
771117,FOODS_3_825_WI_3_0.995_evaluation,5.555872,3.375985,4.318140,2.463651,4.668456,3.931610,3.686286,5.555872,3.375985,...,4.668456,3.931610,3.686286,5.555872,3.375985,4.318140,2.463651,4.668456,3.931610,3.686286
771118,FOODS_3_826_WI_3_0.995_evaluation,3.138254,3.318334,3.010746,3.307220,2.876235,3.464891,5.979320,3.138254,3.318334,...,2.876235,3.464891,5.979320,3.138254,3.318334,3.010746,3.307220,2.876235,3.464891,5.979320


In [56]:
final_pred.to_csv('submission_lgm_final.csv', index=False)